# Warm up
Let's start with some warm up exercises to get you familiar with the database and do some SQL querying.
Make sure to have downloaded the NBA dataset form Kaggle as described in the README.

In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path

## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / 'data/nba.sqlite'
# There seems to be a compatibility issue with the pathy library and the processor.

## Create a SQLite connection and run some SQL queries

In [ ]:
con = sql.connect(DATA_PATH)  # connect to the database
query = 'SELECT * FROM game LIMIT 10'  # write a query that selects the first 10 rows from the game table
top_10_game = pd.read_sql(query, con)  # read the query into a pandas dataframe
top_10_game.head()  # print the dataframe

In [ ]:
# let's look at the schema of the database
# TODO: sqlite_master is the name of the table you want to inspeact: Replace TABLE_NAME with the name of the table you want to inspect
query = """
    SELECT *
    FROM
    sqlite_master
"""

pd.read_sql(query, con).sort_values('name')

In [ ]:
# let's see all the tables we have in the database
# TODO: Filter for tables only
query = """
    SELECT name
    FROM sqlite_master
    WHERE type='table'
"""
pd.read_sql(query, con)

In [ ]:
# let's have a look at the columns in the game table
query = """
    PRAGMA table_info(draft_combine_stats)
"""
pd.read_sql(query, con)

In [ ]:
# let's find the earliest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date
    LIMIT 1
"""

print(pd.read_sql(query, con))

# let's find the latest recorded game
query = """
   PRAGMA table_info(game)
"""

print(pd.read_sql(query, con))


In [ ]:
# 1. Interesting question: Which NBA teams had the highest (home) win percentage over time?
# Answer: We create a "live" updated table that shows the win percentage of each team after each game.


# Drop the table if it already exists
con.execute("DROP TABLE IF EXISTS team_win_statistics")

# Create the table
query = """
CREATE TABLE team_win_statistics AS
SELECT 
    game.team_id_home AS team_id,
    team_details.nickname AS team_name,
    game.game_date AS date,
    SUM(CASE WHEN game.wl_home = 'W' THEN 1 ELSE 0 END) OVER (PARTITION BY game.team_id_home ORDER BY game.game_date) AS wins,
    SUM(CASE WHEN game.wl_home = 'L' THEN 1 ELSE 0 END) OVER (PARTITION BY game.team_id_home ORDER BY game.game_date) AS losses,
    ROUND(
        CAST(SUM(CASE WHEN game.wl_home = 'W' THEN 1 ELSE 0 END) OVER (PARTITION BY game.team_id_home ORDER BY game.game_date) AS FLOAT) /
        NULLIF(SUM(CASE WHEN game.wl_home IN ('W', 'L') THEN 1 ELSE 0 END) OVER (PARTITION BY game.team_id_home ORDER BY game.game_date), 0),
        2
    ) * 100 AS win_percentage
FROM game
JOIN team_details
ON game.team_id_home = team_details.team_id
ORDER BY game.team_id_home, game.game_date;
"""
con.execute(query)
# Read the first 10 rows of the new table
query = """
SELECT *
FROM team_win_statistics
ORDER BY date DESC
LIMIT 10
"""
pd.read_sql(query, con)


In [ ]:

# 1. Interesting question: What is the most successful team in the NBA in terms of wins over time?


# a. Track all wins as a time series
con.execute("DROP TABLE IF EXISTS team_win_statistics") # Drop the table if it already exists
query = """
CREATE TABLE team_win_statistics AS
SELECT 
    game.team_id_home AS team_id,
    game.game_id AS game_id,
    game.game_date AS date,
    CASE WHEN game.wl_home = 'W' THEN 1 ELSE 0 END AS win,
    ROW_NUMBER() OVER (PARTITION BY game.team_id_home ORDER BY game.game_date) AS total_games_played
FROM game

UNION ALL

SELECT 
    game.team_id_away AS team_id,
    game.game_id AS game_id,
    game.game_date AS date,
    CASE WHEN game.wl_away = 'W' THEN 1 ELSE 0 END AS win,
    ROW_NUMBER() OVER (PARTITION BY game.team_id_away ORDER BY game.game_date) AS total_games_played
FROM game

ORDER BY team_id, date;
"""
con.execute(query)

query = """
ALTER TABLE team_win_statistics
ADD COLUMN team_name TEXT;
"""
con.execute(query)

query = """
UPDATE team_win_statistics
SET team_name = (
    SELECT team_details.nickname
    FROM team_details
    WHERE team_details.team_id = team_win_statistics.team_id
);
"""
con.execute(query)


query="""
ALTER TABLE team_win_statistics
ADD COLUMN ma_82 FLOAT;
"""
con.execute(query)
query = """
UPDATE team_win_statistics
SET ma_82 = (
    SELECT ROUND(SUM(win) OVER (
        PARTITION BY team_id
        ORDER BY date
        ROWS BETWEEN 81 PRECEDING AND CURRENT ROW
    ) / 82.0, 2)
);
"""
con.execute(query)


query = """
SELECT *
FROM team_win_statistics
ORDER BY date DESC
LIMIT 100
"""
pd.read_sql(query, con)


In [ ]:
# b. Join the team names (Modify the table to include the team names)


# Show all variables in the game table
query = """
SELECT *    
FROM team_win_statistics
ORDER BY date DESC
LIMIT 100
"""
pd.read_sql(query, con)
con.execute(query)